# Test: graphfw.io.writers.sql_writer.write_sql

Dieses Notebook demonstriert die wichtigsten Optionen:

1. **recreate = True** – Tabelle droppen und aus DF neu anlegen
2. **alignColumns** (inkl. Aliase) – (N)VARCHAR-Spalten verbreitern, wenn DF längere Strings enthält
3. **addcolumns** – neue DF-Spalten via `evolve_on_new_columns=True` hinzufügen
4. **Append** – Daten **anhängen** statt löschen (`truncate=False`)
5. **Truncate** – Tabelle leeren vor Insert (`truncate=True`)

⚠️ Erfordert erreichbaren SQL Server & gültige Zugangsdaten.
Die Beispiele verwenden bewusst eine temporäre Testtabelle.

In [ ]:
from datetime import datetime
import os
import pandas as pd
from sqlalchemy import text
from graphfw.io.writers.sql_writer import build_engine, write_sql

# ---- Verbindung konfigurieren ----
SERVER = os.getenv("SQL_SERVER", "localhost")
DB_NAME = os.getenv("SQL_DB", "BI_RAW")
USER = os.getenv("SQL_USER")
PWD = os.getenv("SQL_PWD")

engine = build_engine({
    "server": SERVER,
    "driver": "ODBC Driver 17 for SQL Server",
    "params": "TrustServerCertificate=yes"
}, db_name=DB_NAME, username=USER, password=PWD)

SCHEMA = os.getenv("SQL_SCHEMA", "stg")
TABLE = f"Test_SP_{datetime.now().strftime('%Y%m%d_%H%M%S')}"  # eindeutiger Tabellenname
TABLE

## 1) Recreate = True
Dropt die Tabelle (falls vorhanden) und legt sie neu an.

In [ ]:
df1 = pd.DataFrame({
    "Id": [1, 2, 3],
    "Name": ["A", "B", "C"]
})
ok, info = write_sql(
    df1,
    engine=engine,
    schema=SCHEMA,
    table=TABLE,
    truncate=False,          # bei recreate nicht nötig
    recreate=True,           # <---
)
print("OK:", ok)
info

## 2) Align Columns (Längen an DF anpassen)

Um das zu demonstrieren, erzeugen wir **manuell** eine Spalte `ShortText VARCHAR(10)` und versuchen anschließend, längere Werte einzufügen.

Hinweis: Die Funktion akzeptiert mehrere Schreibweisen:
- `align_columns=True` (empfohlen)
- `alignColumn=True` (Alias)
- `alighnColumn=True` (vorheriger Tippfehler-Alias)

Der vom Nutzer erwähnte Schreibfehler `alithnColum` **wird ignoriert** und hat keine Wirkung – daher zeigen wir beide Fälle.

In [ ]:
with engine.begin() as conn:
    conn.execute(text(f"""
        IF COL_LENGTH('{SCHEMA}.{TABLE}', 'ShortText') IS NULL
            ALTER TABLE [{SCHEMA}].[{TABLE}] ADD [ShortText] VARCHAR(10) NULL;
    """))

df2 = pd.DataFrame({
    "Id": [4, 5],
    "Name": ["LongName_XXXXXXXXXXXXXXXXXXXX", "Y"],
    "ShortText": ["12345678901", "abcdefghijk"]  # 11 Zeichen > 10
})

# (a) Mit falschem Key (hat KEINE Wirkung)
ok_wrong, info_wrong = write_sql(
    df2,
    engine=engine,
    schema=SCHEMA,
    table=TABLE,
    truncate=False,
    **{"alithnColum": True}   # ignoriert
)
print("OK (falscher Key):", ok_wrong)
info_wrong.get("columns_altered"), info_wrong.get("warnings")


In [ ]:
# (b) Mit korrektem Alias -> Spalte wird verbreitert (ALTER COLUMN)
ok_align, info_align = write_sql(
    df2,
    engine=engine,
    schema=SCHEMA,
    table=TABLE,
    truncate=False,
    alignColumn=True  # Alias für align_columns
)
print("OK (align):", ok_align)
info_align.get("columns_altered")

## 3) Add Columns (neue DF-Spalten automatisch hinzufügen)

Aktiviert mit `evolve_on_new_columns=True`.

In [ ]:
df3 = pd.DataFrame({
    "Id": [6],
    "Name": ["Added"],
    "NewCol": ["neu"]   # neue Spalte
})
ok_add, info_add = write_sql(
    df3,
    engine=engine,
    schema=SCHEMA,
    table=TABLE,
    truncate=False,
    evolve_on_new_columns=True
)
print("OK (add columns):", ok_add)
info_add.get("columns_added")

## 4) Append (Daten anhängen)
Mit `truncate=False` werden Daten nicht gelöscht, sondern angefügt.

In [ ]:
df_append = pd.DataFrame({
    "Id": [7, 8],
    "Name": ["Append1", "Append2"]
})
ok_app, info_app = write_sql(
    df_append,
    engine=engine,
    schema=SCHEMA,
    table=TABLE,
    truncate=False
)
print("OK (append):", ok_app)
info_app["rowcount"]

## 5) Truncate (Tabelle leeren)
Mit `truncate=True` wird vor dem Insert ein `TRUNCATE TABLE` ausgeführt.

In [ ]:
df_trunc = pd.DataFrame({
    "Id": [100],
    "Name": ["AfterTruncate"]
})
ok_tr, info_tr = write_sql(
    df_trunc,
    engine=engine,
    schema=SCHEMA,
    table=TABLE,
    truncate=True
)
print("OK (truncate):", ok_tr)
info_tr["sql"], info_tr["rowcount"]

### Kontrolle: Rows zählen

In [ ]:
with engine.connect() as conn:
    cnt = conn.execute(text(f"SELECT COUNT(*) FROM [{SCHEMA}].[{TABLE}]"))
    print("Rows in Tabelle:", list(cnt)[0][0])